In [1]:
import os
import pandas as pd
import tensorflow as tf
from absl import logging
from modules import components, pipeline
from modules.utils import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

c:\Users\zis\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
merge_dataset("data/movies/movies.csv", "data/ratings/ratings.csv")

'Merge dataset success'

In [3]:
PIPELINE_NAME = "cbf_pipeline"

# pipeline inputs
DATA_ROOT = "data/merge"
TRANSFORM_MODULE_FILE = "modules/cbf_transform.py"
TUNER_MODULE_FILE = "modules/cbf_tuner.py"
TRAINER_MODULE_FILE = "modules/cbf_trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


In [4]:
components = components.init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    train_steps=1000,
    eval_steps=500,
    serving_model_dir=os.path.join(
        serving_model_dir, "cbf_model"),
    epochs=5,
)

pipeline = pipeline.init_pipeline(
    pipeline_root=pipeline_root,
    pipeline_name=PIPELINE_NAME,
    metadata_path=metadata_path,
    components=components,
)

BeamDagRunner().run(pipeline)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\cbf_pipeline\Transform\transform_graph\40\.temp_path\tftransform_tmp\5cba45d81ad74f059ae3c8fff6ba84d7\assets


INFO:tensorflow:Assets written to: outputs\cbf_pipeline\Transform\transform_graph\40\.temp_path\tftransform_tmp\5cba45d81ad74f059ae3c8fff6ba84d7\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movieid_xf (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 genres_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 userid_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 2)            0           ['movieid_xf[0][0]',             
                                                                  'genres_xf[0][0]']          

ERROR:absl:Execution 42 failed.


ValueError: in user code:

    File "c:\Users\zis\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\zis\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\zis\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\zis\anaconda3\lib\site-packages\keras\engine\training.py", line 995, in train_step
        self._validate_target_and_loss(y, loss)
    File "c:\Users\zis\anaconda3\lib\site-packages\keras\engine\training.py", line 949, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.MeanSquaredError object at 0x00000220C20992E0>, and therefore expects target data to be provided in `fit()`.